In [ ]:
#validationあるなしによる速度比較
import time
from typing import List
from pydantic import BaseModel


# ==== モデル定義 ====
class Address(BaseModel):
    street: str
    city: str
    country: str
    postal_code: str


class User(BaseModel):
    id: int
    name: str
    email: str
    age: int
    is_active: bool
    address: Address
    tags: List[str]


# ==== テストデータ生成 ====
def make_data(n: int):
    data = []
    for i in range(n):
        data.append(
            {
                "id": i,
                "name": f"User{i}",
                "email": f"user{i}@example.com",
                "age": 20 + i % 30,
                "is_active": bool(i % 2),
                "address": {
                    "street": f"Street {i}",
                    "city": "Tokyo",
                    "country": "Japan",
                    "postal_code": f"{10000+i}",
                },
                "tags": [f"tag{j}" for j in range(5)],
            }
        )
    return data


# ==== 計測関数 ====
def benchmark(data, mode: str):
    start = time.perf_counter()

    if mode == "validate":
        # 通常のバリデーションあり
        objs = [User.model_validate(item) for item in data]

    elif mode == "construct":
        # バリデーション完全スキップ（高速）
        # model_construct は内部チェックも行わないので、信頼できるデータ専用
        objs = [User.model_construct(**item) for item in data]

    elif mode == "validate_unchecked":
        # 比較用: バリデーションを最小限にする（strict=False）
        objs = [User.model_validate(item, strict=False) for item in data]

    else:
        raise ValueError("mode must be 'validate' or 'construct'")

    elapsed = time.perf_counter() - start
    print(f"{mode:<15}: {elapsed:.6f} 秒")
    return objs



n = 1_000_000
data = make_data(n)

    # ウォームアップ
benchmark(data[:1000], "validate")
benchmark(data[:1000], "construct")

    # 本番測定
benchmark(data, "validate")
benchmark(data, "construct")
benchmark(data, "validate_unchecked")


validate       : 0.003060 秒
construct      : 0.002251 秒
validate       : 7.230892 秒
construct      : 4.624880 秒
validate_unchecked: 6.506868 秒


[User(id=0, name='User0', email='user0@example.com', age=20, is_active=False, address=Address(street='Street 0', city='Tokyo', country='Japan', postal_code='10000'), tags=['tag0', 'tag1', 'tag2', 'tag3', 'tag4']),
 User(id=1, name='User1', email='user1@example.com', age=21, is_active=True, address=Address(street='Street 1', city='Tokyo', country='Japan', postal_code='10001'), tags=['tag0', 'tag1', 'tag2', 'tag3', 'tag4']),
 User(id=2, name='User2', email='user2@example.com', age=22, is_active=False, address=Address(street='Street 2', city='Tokyo', country='Japan', postal_code='10002'), tags=['tag0', 'tag1', 'tag2', 'tag3', 'tag4']),
 User(id=3, name='User3', email='user3@example.com', age=23, is_active=True, address=Address(street='Street 3', city='Tokyo', country='Japan', postal_code='10003'), tags=['tag0', 'tag1', 'tag2', 'tag3', 'tag4']),
 User(id=4, name='User4', email='user4@example.com', age=24, is_active=False, address=Address(street='Street 4', city='Tokyo', country='Japan', po